In [ ]:
library(tidyverse)
library(DBI)
library(dbplyr)
library(RSQLite)
library(carData)
library(datasets)

### Task 1:

- You have to have at least two tables and there must be at least one column in each of the tables that a relation can be created through joins.
- Create a new connection object by creating a new SQLite db file
- Write your tables into the SQLite database through the connection
- Create lazy query objects to each of the tables in the database (not the original tables, but to the tables imported into the SQLite database)
- Create other lazy query object(s) by using select, filter, mutate, group_by, summarise and any join function, using each of them at least once.
- There should be a final lazy query object that combines all previous steps into one.
- Show the underlying sql query of that object
- Collect the data into a data.frame/tibble/data.table and preview it by calling the name of the assigned object

In [ ]:
state_data <- as.data.frame(state.x77)
usarrests_data <- as.data.frame(USArrests)

In [ ]:
head(state_data)

In [ ]:
head(usarrests_data)

In [ ]:
if (file.exists("~titrek_database1.db")) file.remove("~/titrek_database1.db")
consq <- dbConnect(RSQLite::SQLite(), dbname = "titrek_database1.db")

In [ ]:
# Create tables in the SQLite database
dbWriteTable(consq,
             name = "state_data",
             value = state_data,
             row.names = TRUE)

dbWriteTable(consq,
             name = "usarrests_data",
             value = usarrests_data,
             row.names = TRUE)

In [ ]:
# Create lazy query objects for each table
statedata_tbl <- tbl(consq, "state_data")
usarrests_data_tbl <- tbl(consq, "usarrests_data")

In [ ]:
statedata_tbl

In [ ]:
usarrests_data_tbl

In [ ]:
#select, filter, mutate, group_by, summarise and any join function, using each of them at least once.


In [ ]:
arrest_query1 <- usarrests_data_tbl %>% 
    select(row_names, Murder, Assault)

arrest_query2 <- arrest_query1 %>%
    filter( (Murder>5) & (Assault>235) )
    
arrest_query3 <- arrest_query2 %>%
    mutate(TotalCrime = Murder + Assault)

arrest_query4 <- arrest_query3 %>%
  mutate(CrimeLevel = case_when(
    TotalCrime > 300 ~ "High Crime",
    TotalCrime <= 300 ~ "Low Crime"
  ))

arrest_query5 <- arrest_query4 %>%
  group_by(row_names, CrimeLevel) %>%
  summarise(avg_crime = mean(TotalCrime))

arrest_query5_with_state <- left_join(arrest_query5,
                                      statedata_tbl,
                                      by = "row_names")

arrest_query5_with_state

In [ ]:
arrest_query5_with_state %>%
    show_query

In [ ]:
final_data <- collect(arrest_query5_with_state)
head(final_data)

## Task 2:

- Create a datetime (POSIXct) or date object in any way you like. It can be a simple vector. You can also use a dataset with such a column.

- Use at least three datetime/date operations/functions as to your liking on this datetime/date object.

In [ ]:
datetime_vector <- as.POSIXct(c("2024-03-22 15:00:00",
                                "1999-05-15 00:30:00",
                                "2000-12-24 15:45:00"))

datetime_vector

In [ ]:
hour_component <- format(datetime_vector, "%H%s")
hour_component

In [ ]:
next_day <- datetime_vector + 86400 
next_day

In [ ]:
time_diff <- as.numeric(difftime(Sys.time(),
                                 datetime_vector,
                                 units = "mins"))
time_diff